<a href="https://colab.research.google.com/github/tathianamb/widat/blob/main/widat_handson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
%%capture
! sudo apt update && sudo apt install pciutils lshw

In [18]:
%%capture
!curl -fsSL https://ollama.com/install.sh | sh

In [19]:
%%capture
!nohup ollama serve > ollama.log 2>&1 &

In [20]:
%%capture
! ollama pull gemma3:4b

In [21]:
%%capture
!ollama pull nomic-embed-text

In [22]:
%%capture
! pip install langchain langchain-community faiss-cpu ollama

In [11]:
import ollama
import numpy as np
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import warnings
warnings.filterwarnings("ignore")

class ImprovedDogRAG:
    def __init__(self):
        print("🐕 Inicializando Sistema RAG Melhorado...")

        self.client = ollama.Client()
        self.embeddings = OllamaEmbeddings(model="nomic-embed-text")

        # Configuração do text splitter
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=300,
            chunk_overlap=50,
            separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""]
        )

        # Cria base de conhecimento detalhada
        self.create_knowledge_base()
        self.create_vector_store()

        print("✅ Sistema RAG pronto!")

    def create_knowledge_base(self):
        """Cria base de conhecimento estruturada"""
        self.knowledge_base = [
            {
                "title": "Quantidade de Ração por Peso",
                "content": """
                QUANTIDADE DE RAÇÃO DIÁRIA POR PESO DO CÃO:

                CÃES PEQUENOS (até 10kg):
                - 1/2 a 1 xícara de ração por dia
                - Dividir em 2-3 refeições

                CÃES MÉDIOS (10-25kg):
                - 1 a 2 xícaras de ração por dia
                - Dividir em 2 refeições

                CÃES GRANDES (25-45kg):
                - 2 a 4 xícaras de ração por dia
                - Dividir em 2 refeições

                CÃES GIGANTES (acima de 45kg):
                - 4 a 6 xícaras de ração por dia
                - Dividir em 2-3 refeições menores

                OBSERVAÇÃO: Sempre verificar a tabela do fabricante da ração e ajustar conforme atividade física do animal.
                """
            },
            {
                "title": "Frequência de Alimentação",
                "content": """
                FREQUÊNCIA DE REFEIÇÕES POR IDADE:

                FILHOTES (2-4 meses):
                - 4 refeições por dia
                - Ração específica para filhotes
                - Quantidade: seguir tabela do fabricante

                FILHOTES (4-6 meses):
                - 3 refeições por dia
                - Transição gradual para ração adulta

                FILHOTES (6-12 meses):
                - 2-3 refeições por dia
                - Ração para filhotes ou adultos jovens

                ADULTOS (1-7 anos):
                - 2 refeições por dia
                - Ração para adultos
                - Horários fixos (manhã e tarde/noite)

                SENIORES (7+ anos):
                - 2 refeições menores por dia
                - Ração específica para idosos
                - Digestão mais lenta
                """
            },
            {
                "title": "Alimentos Tóxicos para Cães",
                "content": """
                ALIMENTOS PROIBIDOS PARA CÃES:

                EXTREMAMENTE TÓXICOS:
                - Chocolate (principalmente amargo)
                - Uvas e passas
                - Cebola e alho
                - Abacate
                - Xilitol (adoçante)
                - Álcool
                - Cafeína

                PERIGOSOS:
                - Ossos cozidos (podem quebrar)
                - Nozes macadâmia
                - Frutas com caroço (pêssego, ameixa)
                - Massa de pão crua
                - Sal em excesso
                - Alimentos muito condimentados

                EM CASO DE INGESTÃO: Procure veterinário imediatamente.
                """
            },
            {
                "title": "Calendário de Vacinação",
                "content": """
                PROTOCOLO BÁSICO DE VACINAÇÃO:

                PRIMEIRA VACINAÇÃO (6-8 semanas):
                - V8 ou V10 (múltipla)
                - Protege contra: cinomose, hepatite, parainfluenza, parvovirose, etc.

                SEGUNDA VACINAÇÃO (10-12 semanas):
                - V8 ou V10 (reforço)
                - Intervalo de 3-4 semanas da primeira

                TERCEIRA VACINAÇÃO (14-16 semanas):
                - V8 ou V10 (reforço final)
                - Antirrábica (primeira dose)

                REFORÇO ANUAL:
                - V8 ou V10 (anual)
                - Antirrábica (anual)

                VACINAS OPCIONAIS:
                - Leishmaniose (3 doses)
                - Giardíase
                - Gripe canina

                IMPORTANTE: Não sair na rua até completar protocolo.
                """
            },
            {
                "title": "Exercícios por Raça",
                "content": """
                NECESSIDADES DE EXERCÍCIO POR TIPO DE CÃO:

                RAÇAS DE BAIXA ENERGIA (30-45 minutos/dia):
                - Bulldog Inglês e Francês
                - Pug
                - Basset Hound
                - Shih Tzu
                - Pequinês
                Atividades: caminhadas curtas, brincadeiras leves

                RAÇAS DE ENERGIA MODERADA (60-90 minutos/dia):
                - Golden Retriever
                - Labrador
                - Beagle
                - Cocker Spaniel
                - Setter
                Atividades: caminhadas longas, natação, buscar bola

                RAÇAS DE ALTA ENERGIA (2+ horas/dia):
                - Border Collie
                - Pastor Alemão
                - Husky Siberiano
                - Jack Russell Terrier
                - Weimaraner
                Atividades: corrida, trilhas, agility, exercícios mentais

                DICAS IMPORTANTES:
                - Evitar exercícios 1-2 horas após refeições
                - Filhotes: 5 minutos por mês de idade, 2x ao dia
                - Dias quentes: exercitar no início da manhã ou final da tarde
                """
            },
            {
                "title": "Sinais de Emergência",
                "content": """
                SITUAÇÕES DE EMERGÊNCIA VETERINÁRIA:

                PROCURE VETERINÁRIO IMEDIATAMENTE:
                - Dificuldade respiratória severa
                - Convulsões ou tremores incontroláveis
                - Vômitos com sangue
                - Diarreia com sangue ou muito aquosa
                - Abdômen inchado e rígido (torção gástrica)
                - Temperatura acima de 40°C ou abaixo de 37°C
                - Ferimentos profundos
                - Suspeita de envenenamento
                - Não consegue urinar
                - Colapso ou desmaio

                AGENDAR CONSULTA URGENTE:
                - Perda de apetite por mais de 24 horas
                - Letargia extrema
                - Vômitos frequentes (sem sangue)
                - Diarreia persistente
                - Dificuldade para defecar
                - Mudanças drásticas no comportamento
                - Claudicação persistente

                PARÂMETROS NORMAIS:
                - Temperatura: 38-39°C
                - Frequência cardíaca: 60-120 bpm
                - Respiração: 10-30 por minuto (em repouso)
                """
            },
            {
                "title": "Higiene e Cuidados",
                "content": """
                ROTINA DE HIGIENE CANINA:

                BANHOS:
                - Cães de pelo curto: a cada 4-6 semanas
                - Cães de pelo longo: a cada 2-3 semanas
                - Cães com problemas de pele: conforme veterinário
                - Usar sempre shampoo específico para cães
                - Água morna, enxaguar bem

                ESCOVAÇÃO:
                - Pelo curto: 2-3 vezes por semana
                - Pelo longo: diariamente
                - Pelo crespo: diariamente com produtos específicos
                - Remove pelos mortos e previne nós

                CUIDADOS ESPECÍFICOS:
                - Dentes: escovação 2-3 vezes por semana
                - Unhas: corte mensal ou quando fazem ruído
                - Orelhas: limpeza semanal com produto específico
                - Olhos: limpeza diária se necessário (raças braquicefálicas)

                SINAIS DE ALERTA:
                - Mau odor persistente
                - Coceira excessiva
                - Vermelhidão ou irritação
                - Secreções anormais
                """
            }
        ]

        print(f"📚 Base de conhecimento: {len(self.knowledge_base)} documentos")

    def create_vector_store(self):
        """Cria vector store otimizado"""
        all_documents = []

        # Processa cada documento da base
        for doc_data in self.knowledge_base:
            # Documento completo
            full_doc = Document(
                page_content=f"{doc_data['title']}\n\n{doc_data['content']}",
                metadata={
                    "title": doc_data["title"],
                    "type": "full_document"
                }
            )
            all_documents.append(full_doc)

            chunks = self.text_splitter.split_text(doc_data['content'])
            for i, chunk in enumerate(chunks):
                chunk_doc = Document(
                    page_content=f"Sobre {doc_data['title']}: {chunk}",
                    metadata={
                        "title": doc_data["title"],
                        "type": "chunk",
                        "chunk_id": i
                    }
                )
                all_documents.append(chunk_doc)

            title_doc = Document(
                page_content=doc_data['title'],
                metadata={
                    "title": doc_data["title"],
                    "type": "title_only"
                }
            )
            all_documents.append(title_doc)

        self.vector_store = FAISS.from_documents(all_documents, self.embeddings)
        print(f"📊 Índice criado com {len(all_documents)} documentos/chunks")

    def search_knowledge(self, query: str, k: int = 5):
        """Busca melhorada com múltiplas estratégias"""
        docs = self.vector_store.similarity_search(query, k=k)

        seen_titles = set()
        unique_docs = []

        for doc in docs:
            title = doc.metadata.get('title', '')
            if title not in seen_titles:
                seen_titles.add(title)
                unique_docs.append(doc)

        return unique_docs[:3]

    def generate_answer(self, question: str):
        """Geração de resposta melhorada"""

        relevant_docs = self.search_knowledge(question)

        if relevant_docs:

            context = "INFORMAÇÕES RELEVANTES DA BASE DE CONHECIMENTO:\n\n"
            for i, doc in enumerate(relevant_docs, 1):
                title = doc.metadata.get('title', 'Documento')
                context += f"{i}. {title}\n{doc.page_content}\n\n"

            print(f"📚 Encontrados {len(relevant_docs)} documentos relevantes:")
            for doc in relevant_docs:
                print(f"   • {doc.metadata.get('title', 'Sem título')}")
        else:
            context = "Nenhuma informação específica encontrada na base de conhecimento."
            print("ℹ️ Nenhum documento específico encontrado, usando conhecimento geral")

        system_prompt = """Você é um veterinário especialista em cuidados caninos.
        Use PRIORITARIAMENTE as informações da base de conhecimento fornecida.
        Se as informações não forem suficientes, complemente com conhecimento geral, mas sempre indique isso.
        Seja prático, educado e direto. Sempre recomende consultar veterinário para casos específicos."""

        user_prompt = f"""{context}

PERGUNTA: {question}

Responda de forma clara e prática, usando as informações fornecidas:"""

        try:
            response = self.client.chat(
                model='gemma3:4b',
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                options={
                    "temperature": 0.7,
                    "num_predict": 400,
                    "top_p": 0.9,
                    "stop": ["PERGUNTA:", "\n\nPERGUNTA:"]
                }
            )

            return response['message']['content'].strip()

        except Exception as e:
            return f"❌ Erro ao gerar resposta: {e}"

In [23]:
def run_chatbot():
    """Executa chat melhorado"""
    try:
        print("🔄 Inicializando sistema RAG melhorado...")
        rag = ImprovedDogRAG()

        print("\n" + "="*60)
        print("🐕 ASSISTENTE VETERINÁRIO RAG MELHORADO")
        print("="*60)
        print("Comandos especiais:")
        print("• 'sair' - encerrar chat")
        print("• 'topics' - ver tópicos disponíveis")
        print("• 'test' - executar perguntas de teste")
        print("="*60)

        while True:
            question = input("\n🐕 Você: ").strip()

            if not question:
                continue

            print(f"\n🤖 Assistente: ", end="")
            answer = rag.generate_answer(question)
            print(answer)

    except Exception as e:
        print(f"❌ Erro: {e}")

In [ ]:
run_chatbot()

🔄 Inicializando sistema RAG melhorado...
🐕 Inicializando Sistema RAG Melhorado...
📚 Base de conhecimento: 7 documentos
📊 Índice criado com 44 documentos/chunks
✅ Sistema RAG pronto!

🐕 ASSISTENTE VETERINÁRIO RAG MELHORADO
Comandos especiais:
• 'sair' - encerrar chat
• 'topics' - ver tópicos disponíveis
• 'test' - executar perguntas de teste

🐕 Você: eu quero saber sobre alimentação do cachorro

🤖 Assistente: 📚 Encontrados 2 documentos relevantes:
   • Frequência de Alimentação
   • Alimentos Tóxicos para Cães
Olá! Como veterinário especialista em cuidados caninos, posso te dar algumas orientações sobre alimentação do seu cão, baseadas nas informações que tenho.

**Frequência de Alimentação:**

A frequência ideal de alimentação para cães varia de acordo com a idade, tamanho e nível de atividade. Filhotes geralmente precisam de 3-4 refeições por dia, enquanto cães adultos podem se beneficiar de duas refeições diárias.

**Alimentos Tóxicos:**

É crucial estar atento aos alimentos que pode